In [1]:
%matplotlib inline
import pennylane as qml
from pennylane import numpy as np
import pandas as pd

![problem graph](./assets/graph.png)

In [117]:
idx = pd.Index(['A', 'B', 'C', 'D', 'E'])
df = pd.DataFrame([
    np.array([0, 5.8, 5.8, 7.6, 8.8], dtype=np.float16),
    np.array([5.8, 0, 2.4, 4.3, 4.3], dtype=np.float16),
    np.array([5.8, 2.4, 0, 3.5, 4.7], dtype=np.float16),
    np.array([7.6, 4.3, 3.5, 0, 1.4], dtype=np.float16),
    np.array([8.8, 4.3, 4.7, 1.4, 0], dtype=np.float16),
    ], 
    columns=['A', 'B', 'C', 'D', 'E'], 
    index=idx)
df = df.style.set_caption('Points Distances')
df

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.8,0.0,2.4,4.3,4.3
C,5.8,2.4,0.0,3.5,4.7
D,7.6,4.3,3.5,0.0,1.4
E,8.8,4.3,4.7,1.4,0.0



Your are at the point `A`, then you need to choose between point (`B`, `C`, `D` or `E`). However, you can pass though points more than once.

so you can take: `A - B - C - B - D - E`, as a valid path, for example.

You must keep in mind that the point `A` is the `start and end point`.

---

# Mapping points distances in time

---

# Code


In [3]:
Q = df.to_numpy()
Q

array([[0. , 5.8, 5.8, 7.6, 8.8],
       [5.3, 0. , 2.4, 4.3, 4.3],
       [7.1, 1.7, 0. , 3.5, 4.7],
       [8.7, 4. , 3.4, 0. , 1.4],
       [9.8, 5.1, 4.5, 1.4, 0. ]])

In [72]:
def create_hamiltonian(Q:np.array, P:np.int8, P_constant:np.int8):
    print("Q MATRIX:")
    print(Q)
    
    w,h = Q.shape
    print(f"dimensions: w={w}, h={h}\n")

    s_factor = lambda i: (1 + qml.PauliZ(i))/2
    
    H = (9 * P) * qml.Identity(1)
    for i in range(w):
        for j in range(h):
            if(i == j):
                continue

            val = Q[i][j]
            print(f'i={i},j={j},val={val}')

            H += val*s_factor(i)@s_factor(j)

    # Penality
    print("\n---Penality---\n")
    mapping_values = [*list(range(w)), P_constant]
    for i, i_val in enumerate(mapping_values):
        partial = None
        if(i < w):
            partial = P * s_factor(i) @ s_factor(i)

        for j, j_val in enumerate(mapping_values):
            if(i == j):
                continue

            print(f'i={i},j={j}')
            
            if(partial is None):
                partial = P * i_val * s_factor(j)
            elif(i >= w):
                partial += P * i_val * s_factor(j)
            else:
                partial += P * s_factor(i) @ s_factor(j)
                
        
        H += partial    

    return H

In [106]:
H = create_hamiltonian(Q, 20, 4)
H

Q MATRIX:
[[0.  5.8 5.8 7.6 8.8]
 [5.3 0.  2.4 4.3 4.3]
 [7.1 1.7 0.  3.5 4.7]
 [8.7 4.  3.4 0.  1.4]
 [9.8 5.1 4.5 1.4 0. ]]
dimensions: w=5, h=5

i=0,j=1,val=5.8
i=0,j=2,val=5.8
i=0,j=3,val=7.6
i=0,j=4,val=8.8
i=1,j=0,val=5.3
i=1,j=2,val=2.4
i=1,j=3,val=4.3
i=1,j=4,val=4.3
i=2,j=0,val=7.1
i=2,j=1,val=1.7
i=2,j=3,val=3.5
i=2,j=4,val=4.7
i=3,j=0,val=8.7
i=3,j=1,val=4.0
i=3,j=2,val=3.4
i=3,j=4,val=1.4
i=4,j=0,val=9.8
i=4,j=1,val=5.1
i=4,j=2,val=4.5
i=4,j=3,val=1.4

---Penality---

i=0,j=1
i=0,j=2
i=0,j=3
i=0,j=4
i=0,j=5
i=1,j=0
i=1,j=2
i=1,j=3
i=1,j=4
i=1,j=5
i=2,j=0
i=2,j=1
i=2,j=3
i=2,j=4
i=2,j=5
i=3,j=0
i=3,j=1
i=3,j=2
i=3,j=4
i=3,j=5
i=4,j=0
i=4,j=1
i=4,j=2
i=4,j=3
i=4,j=5
i=5,j=0
i=5,j=1
i=5,j=2
i=5,j=3
i=5,j=4


(
    180 * I(1)
  + (2.9 * (Z(0)
  + 1 * I(0))) @ (0.5 * (Z(1)
  + 1 * I(1)))
  + (2.9 * (Z(0)
  + 1 * I(0))) @ (0.5 * (Z(2)
  + 1 * I(2)))
  + (3.8 * (Z(0)
  + 1 * I(0))) @ (0.5 * (Z(3)
  + 1 * I(3)))
  + (4.4 * (Z(0)
  + 1 * I(0))) @ (0.5 * (Z(4)
  + 1 * I(4)))
  + (2.65 * (Z(1)
  + 1 * I(1))) @ (0.5 * (Z(0)
  + 1 * I(0)))
  + (1.2 * (Z(1)
  + 1 * I(1))) @ (0.5 * (Z(2)
  + 1 * I(2)))
  + (2.15 * (Z(1)
  + 1 * I(1))) @ (0.5 * (Z(3)
  + 1 * I(3)))
  + (2.15 * (Z(1)
  + 1 * I(1))) @ (0.5 * (Z(4)
  + 1 * I(4)))
  + (3.55 * (Z(2)
  + 1 * I(2))) @ (0.5 * (Z(0)
  + 1 * I(0)))
  + (0.85 * (Z(2)
  + 1 * I(2))) @ (0.5 * (Z(1)
  + 1 * I(1)))
  + (1.75 * (Z(2)
  + 1 * I(2))) @ (0.5 * (Z(3)
  + 1 * I(3)))
  + (2.35 * (Z(2)
  + 1 * I(2))) @ (0.5 * (Z(4)
  + 1 * I(4)))
  + (4.35 * (Z(3)
  + 1 * I(3))) @ (0.5 * (Z(0)
  + 1 * I(0)))
  + (2.0 * (Z(3)
  + 1 * I(3))) @ (0.5 * (Z(1)
  + 1 * I(1)))
  + (1.7 * (Z(3)
  + 1 * I(3))) @ (0.5 * (Z(2)
  + 1 * I(2)))
  + (0.7 * (Z(3)
  + 1 * I(3))) @ (0.5 * (Z(4

In [99]:
device = qml.device("default.qubit", wires=H.wires)

In [107]:
@qml.qnode(device)
def optimize_circuit(params):
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires=wire)
    return qml.expval(H)

In [108]:
params = np.random.rand(len(H.wires))
optimizer = qml.AdagradOptimizer(stepsize=0.1)
epochs = 200

for epoch in range(epochs):
    params = optimizer.step(optimize_circuit, params)
    print(f"[*] epoch={epoch} : {params}")
params

[*] epoch=0 : [0.63948635 0.33207701 0.52019977 1.06733178 0.34352972 0.77473518]
[*] epoch=1 : [0.71470909 0.41321939 0.59693025 1.13968773 0.42430263 0.84843876]
[*] epoch=2 : [0.77780996 0.48327597 0.66186997 1.19918435 0.49395826 0.90954384]
[*] epoch=3 : [0.83322964 0.54562017 0.71918679 1.25076813 0.55593428 0.96275342]
[*] epoch=4 : [0.88316437 0.60215457 0.77098854 1.29681508 0.61214454 1.01036298]
[*] epoch=5 : [0.92890051 0.65409377 0.8185301  1.33868742 0.6638019  1.05370445]
[*] epoch=6 : [0.97127295 0.70227397 0.8626362  1.37725453 0.71173785 1.09363625]
[*] epoch=7 : [1.01086251 0.74730171 0.90388626 1.41311293 0.7565539  1.13075229]
[*] epoch=8 : [1.04809404 0.78963423 0.94270746 1.44669392 0.79870281 1.16548516]
[*] epoch=9 : [1.08329031 0.82962677 0.97942644 1.47832198 0.83853599 1.1981624 ]
[*] epoch=10 : [1.11670398 0.86756208 1.01430019 1.50824892 0.87633297 1.22903972]
[*] epoch=11 : [1.14853762 0.90366969 1.04753567 1.53667514 0.91232061 1.25832172]
[*] epoch=12 :

tensor([2.67291048, 2.60469353, 2.64741852, 2.80211991, 2.61836213,
        2.33063564], requires_grad=True)

In [103]:
device2 = qml.device("default.qubit", wires=H.wires, shots=1)

In [104]:
@qml.qnode(device2)
def eval_circuit(params):
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires=wire)
    return qml.sample()

In [109]:
eval_circuit(params)

array([1, 1, 1, 1, 1, 0])